In [4]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.integrate as sci
import time 
import threading
import sys

# the function library
from projectlib import *

## Parameters

In [5]:
class Data:
    def __init__(self, dimension, name, commuters_day, N):
        "Dimension gives the dimension of the system, i.e. number of cells"
        # ------- TO DO ---------
        # Depending on the system this has to be adjusted
        self.dimension = dimension
        
        "Alpha describes the rate of infection"
        self.alpha = 0 # Dimensions day^(-1)

        "Beta describes the recovery rate"
        self.beta = 1/14 # Dimensions day^(-1)

        "p is the probability to die from the disease"
        self.p = 2.64e-2
        
        """
        Commuters is an array describing the commuters from and to the different cells. 
        The structure is: The entries in column i are how many commute from i to the cell of the row.
                          Thus, the entries of the row i describe how many commute from different cells to cell i.
        """

        self.commuters = np.array([np.ones(self.dimension) for i in range(self.dimension)])
        "load the txt with given name"
        file = np.loadtxt(name, delimiter="\t")
        "fill commuters with the entries from the file. Note the break statement so it doesn't go to far"
        i = 0
        for row in file:
            if i >= self.dimension:
                break
            self.commuters[i] = np.asarray(row[0:self.dimension])
            i += 1
        
        "Part of day that commuters are in other cells. This is the same value for all cells"
        self.commuters_day = commuters_day

        "Array of population in every cell"
        # ------- TO DO ---------
        # figure out how to best do this
        self.N = N

        
        
    def commutersFrom(self, cfrom):
        """Function to extract the commuters from a cell

        Args:
            cfrom (integer): the cell 

         Returns:
            array: array containing commuters from cfrom
        """
        # ------- TO DO ---------
        # optimize the algorithm
        "Sorting out the necessary entries into array a."
        a = np.zeros(self.dimension)
        for i in range(self.dimension):
            a[i] = self.commuters[i][cfrom]
            
        return a 
    
    def commutersTo(self, cto):
        """Function to return an array with number of commuters coming to cto from other cells.

        Args:
            cto (integer): the cell to which the commuters travel

        Returns:
            array: array containing the commuters to cto.
        """
        return self.commuters[cto][:]
       
        
        

0.0264


## Functions

In [ ]:
def constantApproach(functions, t, data):
    """Function to calculate the time derivatives of the dynamic SIRD modell with commuters. This function uses the approach of constant coefficients. For further explanations see LaTeX

        Args:
            functions (array): array with the functions that define the system of equations. Structure [S_1, ... , S_n, I_1, .... , I_n, R_1, ... , R_n, D_1, ... , D_n]
            t (float): time of timestep. t=1 is 1 day
            data (class): the class with the relevant data

        Returns:
            array: array containing the derivatives of functions at time t.
    """
    # making array with the effective infected Ieff and the effective population Neff
    Ieff = np.array([effective_infected(data.commutersTo, data.commutersFrom, data.N, i, functions[data.dimensions:2*data.dimensions], data.dimensions) for i in range(data.dimensions)])
    Neff = np.array([effective_population(data.commutersFrom, data.N, i) for i in range(data.dimensions)])

    # initializing the return array with the time derivatives of functions
    dfuncdt = np.zeros(data.dimensions*4)

    # the for loop to fill dfuncdt

    for i in range(data.dimensions):
        # see LaTeX for equations
        # dS/dt
        dfuncdt[i] = (data.commuters_day - 1) * functions[i] * functions[i + data.dimensions] - data.commuters_day * data.alpha * (Neff[i] * functions[i] * Ieff[i] / data.N[i] + functions[i] * data.N[i] * np.sum(data.commutersFrom(i) * Ieff))

        # dI/dt \propto -dSdt
        dfuncdt[i + data.dimensions] = - dfuncdt[i] - data.beta * functions[i + data.dimensions]

        # dR/dt
        dfuncdt[i + 2 * data.dimensions] = (1 - data.p) * data.beta * functions[i + data.dimensions]

        # dD/dt
        dfuncdt[i + 3 * data.dimensions] = data.p * data.beta * functions[i + data.dimensions]

    # returning the derivatives at time t
    return dfuncdt






## Comparing $\alpha$'s